In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import product


In [35]:
a = list(product(np.linspace(0,1,11).round(1),np.linspace(0,1,11).round(1),np.linspace(0,1,11).round(1)))
len(a),a
rtable = {}


In [20]:
dopamine_b1 = np.zeros(1331)
for i,kk in enumerate(a):
    dopamine_b1[i] = np.round(2*kk[0]+0.6*kk[1]+0.1*kk[2],2)

dopamine_b2 = np.zeros(1331)
for i,kk in enumerate(a):
    dopamine_b2[i] = np.round(5*kk[0]+0.1*kk[1]+0.2*kk[2],2)
    
dopamine_b3 = np.zeros(1331)
for i,kk in enumerate(a):
    dopamine_b3[i] = np.round(1*kk[0]+0.1*kk[1]+0.01*kk[2],2)


max_dop = np.zeros(len(a))

for ii,ai in enumerate(a):
    rtable[ai,'bacteria_1'] = dopamine_b1[ii]
    rtable[ai,'bacteria_2'] = dopamine_b2[ii]
    rtable[ai,'bacteria_3'] = dopamine_b3[ii]
    max_dop[ii] = dopamine_b1[ii] +dopamine_b2[ii]+dopamine_b3[ii]

    
max(max_dop),max_dop[-1],max_dop

## conversion of probabilities to population
1. sampling distribution - extract quantities. One unit everytime.
2. one to one mapping between nutrient and dopamine.
3. 

In [17]:
nut_qty = np.random.random(3)
nut_qty,a[342], nut_qty< np.array(a[342])
craving_probability = a[342]
nut_qty, a[342],(nut_qty < craving_probability)*1


(array([0.04054308, 0.1192611 , 0.56899783]),
 (0.2, 0.9, 0.1),
 array([1, 1, 0]))

In [ ]:
b =(tuple(np.random.choice(np.linspace(0,1,11),3).round())),np.random.choice(['bacteria_1','bacteria_2','bacteria_3'])

b,rtable.get(b)

In [32]:
np.array((0,2))

array([0, 2])

### observations
1. When action can be nothing, it slowl

3. Possible actions in one nutrient only includes 0.1 (say, a regular increase in the consumption of alcohol. The probability of consuming that nutrient becomes very high along with highest possible reward state; and potentially reducing the craving or eating probability of other nutrients especially (at least one of them).


In [ ]:
class Bacteria():
    def __init__(self,t):
        self.population_size = np.random.randint(100,200)
        self.type = t

    def growPopulation(self):
        self.population_size = self.population_size**2


class Behavior():
    def __init__(self,organism):
        self.nutrients = (1,0.0,0.0)
        self.organism = organism

    def ingestNutrients(self):
        temp = np.array(self.brain.state)
        self.nutrients = np.random.random(temp.size) < temp
        
    def getIngestedNutrients(self):
        return self.nutrients


class Organism():
    
    def __init__(self,n):
        # self.nutrients = n
        self.brain = Brain(self)
        self.gut = [Bacteria('bacteria_'+str(i+1)) for i in range(3)]
        self.behavior = Behavior(self)
        
    def run(self):
        self.brain.setInititalState()
        for i in range(1000):
            self.brain.takeAction()
            self.behavior.ingestNutrients()
            for j in range(3):
                if self.behavior.nutrients[j]==1:
                    self.gut[j].growPopulation()


act = list(product(np.linspace(0,0.1,2),np.linspace(-0.1,0.1,2),np.linspace(-0.1,0.1,2)))
# act.remove((0.0,0.0,0.0))
act = tuple(act)

    
class Brain():
    
    def __init__(self,organism):
        self.organism = organism
#          check what to name this 
        self.actions = act
        self.state = (0,0,1)
        
        self.alpha = 1
        self.gamma = 0.1
        self.epsilon = 0.01
        self.qtable = {}

    def setInititalState(self):
        self.state = np.random.random(3)
        print("starting with state or craving probabilities :",self.state)

    def takeAction(self):
        all_q = [self.qtable.get((self.state,a),0.0) for a in self.actions]
        all_max_ind = np.where(all_q == np.max(all_q))[0]
        action = self.actions[np.random.choice(all_max_ind)]
        
#         use exploration
        if np.random.random()< self.epsilon:
            action = tuple(np.random.choice([0.1,0.0,-0.1], 3))
            
            
        self.ostate = self.state
        self.oaction = action
        self.state = tuple(np.round(np.clip(np.array(self.ostate) + np.array(action),0,1),1))
        

        # this is the summation of dopamine from all bacterium, say         
        self.oreward = np.sum([rtable.get((self.ostate,b.type)) for b in self.organism.gut])
#         print(""" While selecting an action: 
#         old state = {0}
#         action taken = {1}
#         new state = {2}

#         reward = {3}
#         """.format(self.ostate, action,self.state,self.oreward,self.state))
        
        self.learn()
        
        
    def learn(self):
        old_q = self.qtable.get((self.ostate,self.oaction),0.0)
#         print("""
#         old_q  = {0}
#         self state = {4}, old state = {5}
#         qtable = {3}
#         """.format(old_q,len(self.actions),self.actions,
#                   self.qtable.get((self.state,self.actions[0]),np.nan),
#                    self.state,self.ostate))

        max_new_q = np.max([self.qtable.get((self.state,a),0.0) for a in self.actions])
        
        
        newq = old_q + self.alpha*(self.oreward + self.gamma*max_new_q - old_q)
        
        self.qtable[(self.ostate,self.oaction)] = np.round(newq,3)
        
        
#         print("""
#                 Old Q = {0}
#         max New Q = {1}
#         New Q = {2}

#         Q table = {3}
#     """.format(old_q,max_new_q,newq,self.qtable))

o = Organism((0.2,0.4,0.3))
o.run()
o.brain.qtable

In [ ]:
from collections import OrderedDict
oo = OrderedDict(o.brain.qtable)
oo.keys()

In [ ]:
max_value = max(o.brain.qtable.values())


[k for k,v in o.brain.qtable.items() if v == max_value],max_value

In [ ]:
act = list(product(np.linspace(0,0.1,2),np.linspace(-0.1,0.1,2),np.linspace(-0.1,0.1,2)))
# act.remove((0.0,0.0,0.0))
act = tuple(act)
act

In [ ]:
class QLearn():
    def __init__(self):
        self.alpha = 0.2
        self.gamma = 0.7
        self.epsilon = 0.1
        self.qtable = {}
        self.agent = None
        self.ostate = ()
        self.oaction = 0
        
    def getState(self):
        return self.agent.state
        
    def getBestAction(self):
        all_q = [self.qtable.get((self.getState(),a),0.0) for a in self.agent.actions]
        all_max_ind = np.where(all_q == np.max(all_q))[0]
        action = self.agent.actions[np.random.choice(all_m_ind)]

        print(f"""
        possible q for all actions = {all_q}
        index of all max = {all_max_ind}
        action = {action_taken}
        """)
        
        self.ostate = self.getState()
        self.oaction = action
        return action
        
    def learn(self):
        old_q = self.qtable.get((self.ostate,self.oaction),0.0)
        max_new_q = np.max([self.qtable.get((self.getState(),a),0.0) for a in self.agent.actions])

        newq = old_q + self.alpha*(self.agent.reward + self.gamma*max_new_q - old_q)
        self.qtable[(self.ostate,self.oaction)] = newq
        
        

In [ ]:
# global clock


class Organism():
    def __init__(self):
        self.categories = ['a','b','c']        
        self.gut = [Bacteria(c,self) for c in self.categories]
#         self.eatingBehavior - somehow this shud reflect on different nutrients intake 

        


In [ ]:
# class Bacteria():
#     def __init__(self,cat, org):
#         self.category = cat
#         self.organism = org
#         self.population = np.random.randint(100,200)
# #         self.chemical = output(cat)
        
#     def output(self,cat):
#         chem = 0
#         return chem

## breakdown of steps

1. create a world (person) with 5 (5 is becoming unmanageable for this simple example) gut bacteria, and 5 nutrients?
2. [x] mapping between nutrients and bacteria - and that between bacteria and chemicals. make a table for now. 
3. when we initialise the person, the hunger signal tells the person to eat - with some probability ..
4. Nutrients get injected into the system - and once that is done, RL tells us what action to take .. 
    * state is identified with say, time of the day, gut bacterial population and thus, which will be reflected in the amount of dopamine that will come as a reward
    * amount of dopamine (instead of just presence)can be used as the possible reward state. 
    * action is the increase/decrease in the probability of getting a certain type of nutrients. 
    * for this use case, we inject a major proportion of alcohol or sugar into the system.
    * for now, we can also decide the initial composition of the nutrients and gut bacteria to test the use case in the most abstract form ?
    
  

In [ ]:
from collections import defaultdict
d = defaultdict(dict)

# this way is not extensible at all, but for the momemnt - just stick with it to develop a simple use case,

d['carbs']['a'] = 0.1
d['carbs']['b'] = 1
d['carbs']['c'] = 0.2

d['fats']['a'] = 0.1
d['fats']['b'] = 1
d['fats']['c'] = 0





In [ ]:
# # use dictionary instead if dopamine remains constant and only that chemical brings in reward

# a = [['carbs','a','dopamine',1],
#      ['carbs','b','dopamine',3],
#      ['carbs','c','dopamine',1],
#      ['carbs','d','dopamine',0],
#      ['carbs','e','dopamine',0],
#      ['protein','a','dopamine',0],
#      ['protein','b','dopamine',0],
#      ['protein','c','dopamine',0],
#      ['protein','d','dopamine',0],
#      ['protein','e','dopamine',0],
#      ['fats','a','dopamine',0],
#      ['fats','b','dopamine',0],
#      ['fats','c','dopamine',0]
    
    
#     ]

# tab = pd.DataFrame(columns = {'nutrients','bacterium','chemical','quantity'},index = range(len(a)))

# for i,f in enumerate(a):
#     tab.iloc[i] = a[i]
    
# tab


# This is the start of checking the Q table from Ashwin's code



In [ ]:
class Brain:
    def __init__(self, behavior, **kwargs):

        # Params
        self.alpha = kwargs.get('alpha', 0.1)
        self.gamma = kwargs.get('gamma', 0.05)
        self.epsilon = kwargs.get('epsilon', 0.02)

        # Inputs
        self.stateTransitionMatrix = behavior.stateTransitionMatrix
        self.stateSet = list(product(np.linspace(0,1,5).round(2),
                                            np.linspace(0,1,5).round(2)))
        self.actionSet = list(itertools.product(np.linspace(-0.5,0.5,2),
                                            np.linspace(-0.5,0.5,2),np.linspace(-0.5,0.5,2)))

        self.stm = np.zeros((len(self.stateSet), len(self.actionSet)))

        # To be computed
        self.qTable = np.zeros((len(self.stateSet), len(self.actionSet)))
        self.oldStateIndex = 0
        self.oldActionIndex = 0
        self.currentActionIndex = 0
        self.maxFutureQ = 0
        self.theReward = 0
        self.currentStateIndex = int(0)

     
        
    def initialiseSTM(self):
        import time
        tic = time.perf_counter()

        for si,s in enumerate(states):
            for ai,a in enumerate(actions):
        # perform action (eg: state + action) to get values of newState
                ns = np.clip(np.array(s)+np.array(a),0,1).round(2)
                ns = self.computeNewState(s,a)
        # now find the index of the state with which the newState variables match.
                ind = np.where((states == ns).all(axis=1))[0][0]       
                self.stm[si,ai] = ind

    
        print("States computed ", si + 1, "/",  len(self.stateSet))
        toc = time.perf_counter()
        print(f"Done in {toc - tic:0.4f} seconds")



    def computeNewState(self,state,action):
        ns = np.clip(np.array(s)+np.array(a),0,1).round(2)
        return ns


    def computeReward(self, gutOutput):
        """
        Maps gut output to the RL reward
        """
        self.theReward = gutOutput
              
            

        
    def learnAndAct(self):
        """
            This method updates the Q-Table.
            It also controls the state transitions.
            To get state/action history, we should get the currentStateIndex, currentActionIndex
            but be careful of at what stage we are getting them.
        """
        #First store the just implemented state and action as old state and action.
        
        self.oldStateIndex = self.currentStateIndex
        self.oldActionIndex = self.currentActionIndex
        
        # Then use the existing QTable to choose the best action.
        
        if(random.random() > self.epsilon):
            self.exploit()
        else:
            self.explore()

        # Now we have a new value stored for currentActionIndex.
        # thereafter, we use behavior.stateTransitionMatrix to look up the next state
        
        # self.currentStateIndex = int(behavior.stateTransitionMatrix[self.oldStateIndex,
        #                                                         self.oldActionIndex])

        self.currentStateIndex = self.stm[self.oldStateIndex,self.oldActionIndex]
                                                    
            
        # For the new state, compute best possible action value.
        self.maxFutureQ = max(self.qTable[self.currentStateIndex, :])

        # Update the QTable
        self.qTable[self.oldStateIndex, self.oldActionIndex] = self.qTable[self.oldStateIndex, self.oldActionIndex] + self.alpha*(self.theReward + self.gamma*self.maxFutureQ - self.qTable[self.oldStateIndex, self.oldActionIndex])
        
        
        
           
    def exploit(self):
        
        """ 
            Of all the actions in the current state pick the one with maximum Q-value.
            If there is a tie, pick one of the winners at random.
            
            # self.currentActionIndex = random.choice(np.where(qTable[currentStateIndex, :] ==
            #                                      max(qTable[currentStateIndex, :]))[0])
       
        """
        

        # Get Q-values of the possible actions in current state
        possibleActionValues = self.qTable[self.oldStateIndex, :]
        
        # Get indices of best possible actions
        bestActionIndices = np.where(possibleActionValues == max(possibleActionValues))[0]
        # Pick one at random among the winners
        self.currentActionIndex = random.choice(bestActionIndices)
        
        
    def explore(self):
        
        #Pick a random action and store its index
        self.currentActionIndex = np.random.choice(range(len(self.actionSet)))


In [221]:
import time
tic = time.perf_counter()


# states = list(product(np.linspace(0,1,5).round(2),
#                                             np.linspace(0,1,5).round(2)))
# actions = list(product(np.linspace(-0.25,0.25,3).round(2),np.linspace(-0.25,0.25,3).round(2)))
states = list(product(np.linspace(0,1,11).round(2),np.linspace(0,1,11).round(2),
                                    np.linspace(0,1,11).round(2)))
actions = list(product(np.linspace(-0.1,0.1,2).round(2),np.linspace(-0.1,0.1,2).round(2)
,np.linspace(-0.1,0.1,2).round(2)))

# for s in states:
#     for a in actions:
#         ns = np.clip(np.array(s)+np.array(a),0,1)
        
stm = np.zeros((len(states),len(actions)))

for si,s in enumerate(states):
    for ai,a in enumerate(actions):
        # perform action (eg: state + action) to get values of newState
        ns = np.clip(np.array(s)+np.array(a),0,1).round(2)
        # now find the index of the state with which the newState variables match.
        ind = np.where((states == ns).all(axis=1))[0][0]       
        stm[si,ai] = ind
    
    print("States computed ", si + 1, "/",  len(states))

toc = time.perf_counter()
print(f"Done in {toc - tic:0.4f} seconds")


np.set_printoptions(suppress = True)
stm[:10],states[121],actions[4],states[0]

uted  630 / 1331
States computed  631 / 1331
States computed  632 / 1331
States computed  633 / 1331
States computed  634 / 1331
States computed  635 / 1331
States computed  636 / 1331
States computed  637 / 1331
States computed  638 / 1331
States computed  639 / 1331
States computed  640 / 1331
States computed  641 / 1331
States computed  642 / 1331
States computed  643 / 1331
States computed  644 / 1331
States computed  645 / 1331
States computed  646 / 1331
States computed  647 / 1331
States computed  648 / 1331
States computed  649 / 1331
States computed  650 / 1331
States computed  651 / 1331
States computed  652 / 1331
States computed  653 / 1331
States computed  654 / 1331
States computed  655 / 1331
States computed  656 / 1331
States computed  657 / 1331
States computed  658 / 1331
States computed  659 / 1331
States computed  660 / 1331
States computed  661 / 1331
States computed  662 / 1331
States computed  663 / 1331
States computed  664 / 1331
States computed  665 / 1331
Sta

In [224]:
# FASTEST WAY TO CREATE STATE TRANSITION MATRIX

import time
tic = time.perf_counter()

states = list(product(np.linspace(0,1,11).round(2),np.linspace(0,1,11).round(2),
                                    np.linspace(0,1,11).round(2)))
actions = list(product(np.linspace(-0.1,0.1,2).round(2),np.linspace(-0.1,0.1,2).round(2)
,np.linspace(-0.1,0.1,2).round(2)))


stm = np.zeros((len(states),len(actions)))

for i,(s,a) in enumerate(product(states,actions)):
    # perform action (eg: state + action) to get values of newState
    ns = np.clip(np.array(s)+np.array(a),0,1).round(2)
    # now find the index of the state with which the newState variables match.
    ind = np.where((states == ns).all(axis=1))[0][0]       
    si = i//len(states) 
    ai= i%len(actions)
    stm[si,ai] = ind


toc = time.perf_counter()
print(f"Done in {toc - tic:0.4f} seconds")

    

Done in 6.5744 seconds
